<a href="https://colab.research.google.com/github/adammelancon/LPLElectionMap/blob/main/LPLElectionMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [341]:
import pandas as pd

try:
  import geopandas
except ImportError:
  !pip install geopandas # Install geopandas in google colab

import geopandas as gpd
import folium
import numpy as np

radiuscustom = True
radius = 1  # MILES 
radius1 = False
radius3 = False
radius5 = False
markers = True
RADIUSTOM = 1609 * radius  # Converts to meters for map data

# -----------------Branch Lat/Long Dictionary----------------------
branches = {
  "Main Branch": {
      "lat" : "30.226199925369336",
      "long" : "-92.0202574968979",
      },
    "South Regional": {
        "lat" : "30.166477699227492",
        "long" : "-92.08185654289605",
      },
  "North Regional": {
      "lat" : "30.32789494459831",
      "long" : "-92.05006318605113",
      },
    "East Regional": {
        "lat" : "30.144330020794083",
        "long" : "-91.99601356431116",
      },        
  "West Regional": {
      "lat" : "30.231785999603545",
      "long" : "-92.10665348071744",
      },
    "Chenier Branch": {
        "lat" : "30.246295788904458",
        "long" : "-92.01629399025994",
      },
    "Butler Branch": {
        "lat" : "30.247593599828722",
        "long" : "-92.02636283851278",
      },
    "Duson Branch": {
        "lat" : "30.238051932703772",
        "long" : "-92.18131335165813",
      },
    "Milton Branch": {
        "lat" : "30.105160086556964",
        "long" : "-92.07438885266791",
      },
}

In [342]:
# -----------------Pandas Options----------------------
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 10000)
pd.set_option('display.max_rows', 1000)

In [343]:
# -----------------Shapefile Geometry Data Setup----------------------

# Load the precinct numbers and geometry for the 
# Lafayette precincts in a dataframe
shapefile = gpd.read_file("lft.geojson")
shapefile = shapefile[["precinctid", "geometry"]]
shapefile['precinctid'] = shapefile['precinctid'].astype(int)
shapefile = shapefile.reset_index(drop=True)
# print(shapefile)

In [344]:
# -------------Election Results and Precinct Data Setup------------

# Load the precinct id and Yes/No voting data 
# from secretary of state website
df = pd.read_csv("bp.csv")
df.drop(df.tail(1).index,inplace=True) # Drop last line which has totals
df = df.drop(['Office','Parish', 'Ward'], axis=1) # Drop unused columns

# Make numbers floats and create the [total], [win], and [score] columns.
df['YES'] = df['YES'].astype(float)
df['total'] = df[['YES','NO']].sum(axis=1)
df['total'] = df['total'].astype(float)
df['score'] = (df['YES'] / df['total'])
df['score'] = df['score'] * 100.00
df['score'] = df['score'].astype(float)
df['win'] = df['score'] 

# Assign to [win] column 1 if >= 50% in [score], 
# 0 if >= 49, and 2 for tie at 50% exactly
df['win'] = np.where(df['score'] >= 50.0, 0, 1)
df.loc[df['score'] == 50.0, 'win'] = 2

In [345]:
# ---------------------Merging Datasets----------------------

# Merge the data from election and precinct files
shapefile = shapefile.merge(
    df, right_on='Precinct', 
    left_on='precinctid', 
    how='left'
    )
shapefile = shapefile.fillna(0) # If no data fill with 0
# Make number integers
shapefile['YES'] = shapefile['YES'].astype(int)
shapefile['NO'] = shapefile['NO'].astype(int)
shapefile['Precinct'] = shapefile['Precinct'].astype(int)
shapefile['score'] = shapefile['score'].astype(int)
shapefile['win'] = shapefile['win'].astype(int)

# Set the map type for the geometry 
shapefile = shapefile.set_crs(4326)

In [346]:
# -----------------Map Creation ----------------------

# Create folium map instance zoomed in on Lafayette
mymap = folium.Map(location=[30.22, -92], zoom_start=11)
# Options for the Choropleth map
folium.Choropleth(
    geo_data=shapefile,
    name="choropleth",
    data=shapefile,
    columns=["Precinct","win"],
    key_on = 'feature.properties.precinctid', 
    fill_color="Dark2",
    fill_opacity=0.7,
    line_opacity=1,
    highlight=True,
    overlay=True,
    show=True,
    # bins=bins, 
    # legend_name="0 = Lose\n1 = Win\n2 = Tie",
).add_to(mymap)

In [347]:
# -----------------Extra Map Features----------------------


# Add hover functionality for Precinct and Yes vote %
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = shapefile,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Precinct','score'],
        aliases=['Precinct','Yes %'],
        style=("background-color: white; color: #333333; \
        font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
# Append hover feature to exising map.
mymap.add_child(NIL)
mymap.keep_in_front(NIL)

In [348]:
# -----------------Optional Branch Markers/Radius----------------------

def make_markers():
  for i in branches:
    lat = float(branches[i]['lat'])
    long = float(branches[i]['long'])
    folium.Marker(location=[lat, long ], popup=i, tooltip=i, \
                  icon=folium.Icon(icon="book")).add_to(mymap)


def make_radius_markers():
    for i in branches:
      lat = float(branches[i]['lat'])
      long = float(branches[i]['long'])
      if radiuscustom:
        folium.Circle(
          radius=RADIUSTOM,
          location=[lat, long],
          popup= str(radius) + "Mile Radius",
          color="purple",
          fill=False,
          tooltip=str(radius) + " Mile Radius",
          ).add_to(mymap)

      if radius1:
        folium.Circle(
          radius=1609,
          location=[lat, long],
          popup="1 Mile Radius",
          color="red",
          fill=False,
          tooltip="1 Mile Radius",
          ).add_to(mymap)

      if radius3:    
        folium.Circle(
          radius=4828,
          location=[lat, long],
          popup="3 Mile Radius",
          color="green",
          fill=False,
          tooltip="3 Mile Radius",
          ).add_to(mymap)

      if radius5:
        folium.Circle(
          radius=8046,
          location=[lat, long],
          popup="5 Mile Radius",
          color="yellow",
          fill=False,
          tooltip="5 Mile Radius",
          ).add_to(mymap)
    else:
        pass

In [349]:
# -----------------Final Output----------------------

# Add Branch Markers from branch dictionary above
if markers == True:
  make_markers()

# Add radius markers if requested.
if radiuscustom or radius1 or radius3 or radius5:
    make_radius_markers()

In [350]:
# Save output to HTML file.   
# mymap.save('./yeswinnowin.html')
mymap